<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220223_sm_TPU_kfold_hardvoting_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full training

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 311 kB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 19.8 MB/s 
     |████████████████████████████████| 243 kB 77.9 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 134 kB 79.8 MB/s 
     |████████████████████████████████| 1.1 MB 68.4 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 84.7 MB/s 
     |████████████████████████████████| 144 kB 78.7 MB/s 
     |████████████████████████████████| 895 kB 68.6 MB/s 
     |████████████████████████████████| 6.8 MB 70.5 MB/s 
     |████████████████████████████████| 596 kB 76.3 MB/s 
     |████████████████████████████████| 1.2 MB 71.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 3.2 MB/s 
     |████████████████████████████████| 149.9 

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [5]:
kakao_snli = pd.read_csv('/content/drive/MyDrive/KLUE/snli_1.0_train.ko.tsv', sep='\t', encoding='utf-8')
kakao_snli = kakao_snli[:100000]

PATH =  '/content/drive/MyDrive/KLUE'
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

kakao_dev = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.dev.ko.tsv', sep='\t', encoding='utf-8')
kakao_test = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.test.ko.tsv', sep='\t', encoding='utf-8')
klue_dev = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_dev.json')
klue_train = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_train.json')
temp = pd.read_csv(os.path.join(PATH, 'train_data.csv'), encoding='utf-8')

klue_dev = klue_dev[['premise', 'hypothesis', 'gold_label']]
klue_train = klue_train[['premise', 'hypothesis', 'gold_label']]
klue_dev.rename(columns = {'gold_label':'label'}, inplace=True)
klue_train.rename(columns = {'gold_label':'label'}, inplace=True)

train = pd.concat([kakao_dev,kakao_test, kakao_snli])
train.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)
train = pd.concat([temp, train, klue_dev, klue_train], axis=0)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
159516,159516,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",개발 음식 활용 혜택만 주어진다.,contradiction
159517,159517,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",힐링푸드 전문가에게 혜택이 주어진다.,neutral
159518,159518,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",혜택에는 힐링푸드 전문가 양성 교육 참여가 있다.,entailment
159519,159519,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction


In [6]:
from transformers import AutoTokenizer

checkpoint = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
# train_dataset, eval_dataset = train_test_split(train, test_size=0.2, shuffle=False)

# tokenized_train = tokenizer(
#     list(train_dataset['premise']),
#     list(train_dataset['hypothesis']),
#     return_tensors="pt",
#     max_length=128, # Max_Length = 190
#     padding=True,
#     truncation=True,
#     add_special_tokens=True
# )

# tokenized_eval = tokenizer(
#     list(eval_dataset['premise']),
#     list(eval_dataset['hypothesis']),
#     return_tensors="pt",
#     max_length=128,
#     padding=True,
#     truncation=True,
#     add_special_tokens=True
# )

In [8]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [9]:
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label

# train_label = label_to_num(train['label'].values)
# eval_label = label_to_num(eval_dataset['label'].values)

In [10]:
# train_dataset = BERTDataset(tokenized_train, train_label)
# eval_dataset = BERTDataset(tokenized_eval, eval_label)

In [11]:
# train_dataset[0]

In [12]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [13]:
# def training_function():
#   accelerator = Accelerator()

#   train_dataloader = torch.utils.data.DataLoader(
#                     train_dataset, 
#                     batch_size=16, sampler=train_subsampler, num_workers=2)
#   eval_dataloader = torch.utils.data.DataLoader(
#                     train_dataset,
#                     batch_size=16, sampler=test_subsampler, num_workers=2)

#   config = AutoConfig.from_pretrained(checkpoint)
#   config.num_labels = 3
#   model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

#   optimizer = AdamW(model.parameters(), lr=	1e-5)

#   model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

#   num_epochs = 5
#   num_training_steps = num_epochs * len(train_dataloader)
#   progress_bar = tqdm(range(num_training_steps))
#   lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
#       optimizer=optimizer,
#       num_warmup_steps=1,
#       num_training_steps=num_training_steps,
#   )

#   for epoch in range(num_epochs):
#       train_acc = 0.0
#       test_acc = 0.0

#       model.train()
#       for batch_id, batch in enumerate(train_dataloader):
#           outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
#           loss = F.cross_entropy(outputs[0], batch['labels'])
#           accelerator.backward(loss)

#           optimizer.step()
#           lr_scheduler.step()
#           optimizer.zero_grad()
#           progress_bar.update(1)
#           train_acc += calc_accuracy(outputs.logits, batch['labels'])
#       print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

#       model.eval()
#       for batch_id, batch in enumerate(eval_dataloader):
#         with torch.no_grad():
#             outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

#         test_acc += calc_accuracy(outputs.logits, batch['labels'])
#       print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
#       gc.collect()
#   accelerator.wait_for_everyone()
#   unwrapped_model = accelerator.unwrap_model(model)
#   accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/KLUE/model.pt')

In [14]:
# # kfold = StratifiedKFold(n_splits=5, shuffle=True)

# # Start print
# # print('--------------------------------')

# # K-fold Cross Validation model evaluation
# def training_function():
#   kfold = StratifiedKFold(n_splits=5, shuffle=True)

#   print('--------------------------------')

#   for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
#     # print('')
#     print(f'FOLD {fold}')

#     train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
#     test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

#     train_label = label_to_num(train['label'].values)

#     tokenized_train = tokenizer(
#       list(train['premise']),
#       list(train['hypothesis']),
#       return_tensors="pt",
#       max_length=128, # Max_Length = 190
#       padding=True,
#       truncation=True,
#       add_special_tokens=True
#       )
    
#     train_dataset = BERTDataset(tokenized_train, train_label)

#     # def training_function():
#     accelerator = Accelerator()

#     train_dataloader = torch.utils.data.DataLoader(
#                       train_dataset, 
#                       batch_size=8, sampler=train_subsampler)
#     eval_dataloader = torch.utils.data.DataLoader(
#                       train_dataset,
#                       batch_size=8, sampler=test_subsampler)

#     config = AutoConfig.from_pretrained(checkpoint)
#     config.num_labels = 3
#     model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

#     optimizer = AdamW(model.parameters(), lr=	1e-5)

#     model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

#     num_epochs = 5
#     num_training_steps = num_epochs * len(train_dataloader)
#     progress_bar = tqdm(range(num_training_steps))
#     lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
#         optimizer=optimizer,
#         num_warmup_steps=1,
#         num_training_steps=num_training_steps,
#     )

#     for epoch in range(num_epochs):
#         train_acc = 0.0
#         test_acc = 0.0

#         model.train()
#         for batch_id, batch in enumerate(train_dataloader):
#             outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
#             loss = F.cross_entropy(outputs[0], batch['labels'])
#             accelerator.backward(loss)

#             optimizer.step()
#             lr_scheduler.step()
#             optimizer.zero_grad()
#             progress_bar.update(1)
#             train_acc += calc_accuracy(outputs.logits, batch['labels'])
#         print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

#         model.eval()
#         for batch_id, batch in enumerate(eval_dataloader):
#           with torch.no_grad():
#               outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

#           test_acc += calc_accuracy(outputs.logits, batch['labels'])
#         print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
#         gc.collect()
#     accelerator.wait_for_everyone()
#     unwrapped_model = accelerator.unwrap_model(model)
#     accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/KLUE/model' + str(fold) + '.pt')

In [15]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=16, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=16, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 5
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  accelerator.save(unwrapped_model.state_dict(), '/content/model/model' + str(fold) + '.pt')

In [ ]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True)

# Start print
# print('--------------------------------')

# K-fold Cross Validation model evaluation

kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['premise']),
  list(train['hypothesis']),
  return_tensors="pt",
  max_length=128, # Max_Length = 190
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
  # print('')
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = label_to_num(train['label'].values)

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'cl

  0%|          | 0/4985 [00:00<?, ?it/s]

  0%|          | 0/4985 [00:00<?, ?it/s]

  0%|          | 0/4985 [00:00<?, ?it/s]

...

  0%|          | 0/4985 [00:00<?, ?it/s]

  0%|          | 0/4985 [00:00<?, ?it/s]

  0%|          | 0/4985 [00:00<?, ?it/s]

  0%|          | 0/4985 [00:00<?, ?it/s]

epoch 1 train acc 0.8059177532597793
epoch 1 train acc 0.8022191574724172
epoch 1 train acc 0.8109954864593781
epoch 1 train acc 0.8085506519558676
epoch 1 train acc 0.8049774322968907
epoch 1 train acc 0.8056043129388164
epoch 1 train acc 0.8106820461384152
epoch 1 train acc 0.8020937813440321
epoch 1 test acc 0.86625
epoch 1 test acc 0.8695
epoch 1 test acc 0.8645
epoch 1 test acc 0.874
epoch 1 test acc 0.87275
epoch 1 test acc 0.8565
epoch 1 test acc 0.88075
epoch 1 test acc 0.871


In [ ]:
print(my_variable)

In [ ]:
def count_label(df, label):
  count = 0
  cols = ['label1', 'label2', 'label3', 'label4', 'label5']
  for col in cols:
    if df[col] == label:
      count = count +1
  return count

df['neutral'] = df.apply(count_label(df, label))

In [ ]:
config = AutoConfig.from_pretrained(checkpoint)
config.num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

In [ ]:
accelerator = Accelerator()
model = accelerator.unwrap_model(model)
model.load_state_dict(torch.load('/content/drive/MyDrive/model/model.pt'))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

In [ ]:
dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

output_pred = []
output_prob = []

model, dataloader= accelerator.prepare(model, dataloader)

model.eval()

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            token_type_ids=data['token_type_ids']
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
print(pred_answer)

In [ ]:
def num_to_label(label):
    label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(pred_answer)
print(answer)

In [ ]:
df = pd.DataFrame(answer, columns=['index', 'label'])

df.to_csv('/content/drive/MyDrive/KLUE/submission.csv', index=False)

print(df)